In [1]:
from ultralytics import YOLO

# Load a YOLOv8 model (e.g., YOLOv8n, YOLOv8s, YOLOv8m, YOLOv8l, YOLOv8x)
model = YOLO('yolov8n.pt')  # Load a pretrained YOLOv8n model


100%|██████████| 6.25M/6.25M [00:07<00:00, 834kB/s] 


In [4]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')




image 1/1 C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\train\images\2022-03-26-10-00 005.jpg: 384x640 (no detections), 35.5ms
Speed: 3.5ms preprocess, 35.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Convert json label to txt yolo format

In [10]:
import json
import os

def json_to_yolo(json_path, output_dir, class_map):
    # Load the JSON annotation
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Extract image dimensions dynamically
    try:
        img_width = data['imageWidth']
        img_height = data['imageHeight']
    except KeyError:
        print(f"Error: Missing 'imageWidth' or 'imageHeight' in {json_path}")
        return  # Skip this file if keys are missing

    # Extract annotations
    annotations = data.get('shapes', [])  # Safely fetch 'shapes' key
    if not annotations:
        print(f"Warning: No shapes found in {json_path}")
        return  # Skip if no shapes are found
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Determine the output .txt file name based on JSON file name
    base_name = os.path.splitext(os.path.basename(json_path))[0]
    output_file = os.path.join(output_dir, f"{base_name}.txt")
    
    # Open the .txt file for writing
    with open(output_file, 'w') as f:
        for annotation in annotations:
            label = annotation.get('label', None)
            if label not in class_map:
                continue  # Skip unrecognized labels
            
            class_id = class_map[label]
            
            # Convert polygon points to bounding box
            points = annotation.get('points', [])
            if not points:
                continue  # Skip if points are missing

            x_coords = [p[0] for p in points]
            y_coords = [p[1] for p in points]
            x_min, x_max = min(x_coords), max(x_coords)
            y_min, y_max = min(y_coords), max(y_coords)
            
            # Normalize bounding box coordinates (YOLO format)
            x_center = (x_min + x_max) / 2 / img_width
            y_center = (y_min + y_max) / 2 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height
            
            # Write to the .txt file
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    
    print(f"Converted {json_path} to {output_file}")

# Parameters
json_dir = r"C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\annotations_only_non_and_pollenbee_fixed_final"
output_dir = r"C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\labels"
class_map = {"pollenbee": 1, "nonpollenbee": 0}  # Map class names to class IDs

# Process all JSON files
for json_file in os.listdir(json_dir):
    if json_file.endswith(".json"):
        json_to_yolo(os.path.join(json_dir, json_file), output_dir, class_map)


Converted C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\annotations_only_non_and_pollenbee_fixed_final\2022-03-26-10-00 003.json to C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\labels\2022-03-26-10-00 003.txt
Converted C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\annotations_only_non_and_pollenbee_fixed_final\2022-03-26-10-00 005.json to C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\labels\2022-03-26-10-00 005.txt
Converted C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\annotations_only_non_and_pollenbee_fixed_final\2022-03-26-10-00 012.json to C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\labels\2022-03-26-10-00 012.txt
Converted C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\annotations_only_non_and_pollenbee_fixed_final\2022-03-26-10-00 020.json to C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\labels\2022-03-26-10-00 020.txt
Converted C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\test\annotations_only_non_and_pollenbee_fixed_final\2022-03-26-10-00 022.json to C

In [1]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8l.pt')  # Nano model for quick training

# Train the model
model.train(
    data='data.yaml',
    epochs=50,
    imgsz=240,
    batch=2,
    save=True,
    device='cuda',
    workers=2
)


100%|██████████| 83.7M/83.7M [01:02<00:00, 1.40MB/s]


New https://pypi.org/project/ultralytics/8.3.50 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.38  Python-3.9.20 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8l.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=2, imgsz=240, save=True, save_period=-1, cache=False, device=cuda, workers=2, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\labels\train.cache... 1496 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1496/1496 [00:00<?, ?it/s]
val: Scanning C:\Users\MAY02\Desktop\Bee\PollenbeeDataset\labels\val.cache... 381 images, 0 backgrounds, 0 corrupt: 100%|██████████| 381/381 [00:00<?, ?it/s]


Plotting labels to runs\detect\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs\detect\train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      1.25G      1.922     0.9386     0.9317         68        256: 100%|██████████| 748/748 [01:57<00:00,  6.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 96/96 [00:07<00:00, 12.49it/s]

...

                   all        381      11103      0.918       0.92      0.948      0.652



50 epochs completed in 1.590 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 87.6MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 87.6MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics 8.3.38  Python-3.9.20 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
Model summary (fused): 268 layers, 43,608,150 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 96/96 [00:07<00:00, 12.63it/s]


                   all        381      11103      0.921      0.918      0.948      0.652
          nonpollenbee        381      10796      0.956      0.965      0.984       0.72
             pollenbee        272        307      0.887       0.87      0.913      0.585
Speed: 0.6ms preprocess, 6.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\train7


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C448A52F40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [3]:
from ultralytics import YOLO
import cv2

# Load the YOLO model
model = YOLO('best.pt')  # Replace 'best.pt' with the path to your trained model

# Path to the video file
video_path = '2022-04-08-12-30.mp4'  # Replace with your video path
output_path = 'output_video.mp4'  # Path to save the processed video

# Load the video
cap = cv2.VideoCapture(video_path)

# Get video details
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Process the video frame by frame
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break  # Exit loop when the video ends

    # Perform inference with YOLO
    results = model(frame)

    # Draw the results on the frame
    annotated_frame = results[0].plot()

    # Write the frame to the output video
    out.write(annotated_frame)

    # Optional: Display the video in real-time
    cv2.imshow('YOLO Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Video processing complete. Saved to:", output_path)



0: 160x256 13 nonpollenbees, 73.6ms
Speed: 2.0ms preprocess, 73.6ms inference, 2.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 13 nonpollenbees, 16.6ms
Speed: 2.0ms preprocess, 16.6ms inference, 2.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 13 nonpollenbees, 16.2ms
Speed: 1.0ms preprocess, 16.2ms inference, 2.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 14 nonpollenbees, 16.1ms
Speed: 1.0ms preprocess, 16.1ms inference, 2.1ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 15 nonpollenbees, 16.7ms
Speed: 0.0ms preprocess, 16.7ms inference, 2.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 13 nonpollenbees, 16.2ms
Speed: 2.0ms preprocess, 16.2ms inference, 2.1ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 16 nonpollenbees, 16.7ms
Speed: 2.0ms preprocess, 16.7ms inference, 2.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 14 nonpollenbees, 17.4ms
Speed: 0.0ms preprocess, 